In [1]:
import numpy as np
import torch
import math
from torch import nn
import torch.nn.functional as F

In [2]:
torch.cuda.is_available()

True

In [3]:
torch.device('cuda')

device(type='cuda')

: 

In [ ]:
from open_lm.hf import *
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("TRI-ML/DCLM-1B")
model = AutoModelForCausalLM.from_pretrained("TRI-ML/DCLM-1B")

inputs = tokenizer(["Machine learning is"], return_tensors="pt")

In [1]:
from threading import local


def flash_attention(q,k,v,mask=None,dropout=None,block_size=1024):
    #extractttt info from query tensor
    batch_size,num_heads,seq_len,d_k=q.size()
    #scaling factor for the dot product attention
    scale=1/math.sqrt(d_k)
    #intializee tensors to store final output and atteention weights
    output=torch.zeros_like(q)
    attention_weights=torch.zeros(batch_size,seq_len,seq_len,device=q.device)

    #itterate over the sequence in block s
    for block_start in range(0,seq_len,block_size):
        #caclulate the end of the block
        block_end=min(block_start+block_size,seq_len)
        
        #extracl block for query key and values 
        local_q=q[::,:,block_start:block_end]
        local_k=k[:,:,:block_end]
        local_v=v[:,:,:block_end]

        #local attention scaled dod product fo it is calcualted
        local_attention=torch.matmul(local_q,local_k.transpose(-1,-2))*scale

        #apply mask if provided
        if mask is not None:
            local_attn=local_attn+mask[:,:,block_start:block_end,:block_end]

        #apply softmax
        local_attn=F.softmax(local_attn,dim=-1)

        #apply dropout if provided 
        if dropout is not None:
            local_attn=dropout(local_attn)

        #compute output of current block
        local_output=torch.matmul(local_attn,local_v)

        #store output of current block
        output[:,:,block_start:block_end]=local_output
        attention_weights[:,:,block_start:block_end]=local_attn

    return output,attention_weights    

def flash_scaled_dot_product(q,k,v,mask):
    d_k=q.size()[:-1]
    values,attention=flash_attention(q,k,v,mask)
    return values,attention


: 

In [ ]:
class rotaryembedding(nn.Module):
    def __init__(self,dim):
        super().__init__()
        self.dim=dim
    def forward(self,seq_len):
        half_dim=self.dim//2
        emb=math.log(10000)/(half_dim-1)
        emb=torch.exp(torch.arange(half_dim,device=self.device)*-emb)    
